In [ ]:
!pip install heapdict

In [ ]:
# Colab only: Run this cell to download/install packages
import sys

# Graph Library


In this problem, you're going to write a (very minimal) graph library, which uses both the adjacency dictionary and the sparse adjacency matrix representation of a graph.  Using these two representations, you'll implement two of the more well-known large-scale graph processing algorithms: Dijkstra's algorithm for finding single-source shortest paths in the graph, and the PageRank algorithm for determining the importance of nodes in the network.

### The `wikipedia_small` graph

The graph we'll be focusing on for most of this assignment is a directed graph that represents the page links in the English language Wikipedia.  Specifically, we took the (pre-processed) Wikipedia dump from `http://haselgrove.id.au/wikipedia.htm`, which were taken from a 2008 version of Wikipedia, and we then subselected only those nodes that had at least _500 incoming links_.  This resulted in a graph with about 24,000 nodes and 6,000,000 edges.

*Note*: The `http://haselgrove.id.au/wikipedia.htm` has since died off. See Wikipedia itself for information on accessing data: https://en.wikipedia.org/wiki/Wikipedia:Database_download#Where_do_I_get_it?

There are two files included with this notebook that are relevant here:

- `wikipedia_small.graph.gz`
- `wikipedia_small.nodes.gz`
    
The `.graph.gz` file contains a (gzipped) list of integers, two per line.  If the line "`i j`" appears in the file, this indicates a directed edge from node `i` to node `j`.  The `.nodes.gz` file then contains a (gzipped) list of each node the graph, where the link number indicates the node index.  This is how we can relate the node numbers in the `.graph.gz` file to actual pages on Wikipedia.

## Your own Graph class

In the main portion of this assignment, you'll create your own Graph class that mimics some of the functionality of networkx. We'll provide you with some scaffolding and support code.

In [ ]:
import numpy as np
import scipy.sparse as sp
import heapdict
import gzip

# Utility function to read the graph
def read_graph(basename="wikipedia_small"):
    with gzip.open(f"{basename}.nodes.gz", 'rt', encoding="utf-8") as f:
        nodes = [a.strip() for a in f]
    with gzip.open(f"{basename}.graph.gz", 'rt', encoding="utf-8") as f:
        links = []
        for row in f:
            i, j = tuple(row.strip().split())
            links.append((nodes[int(i)], nodes[int(j)]))
    return links

### Q1: Implement `add_edges`

Here is the template for your Graph class.

Note that `self.edges` should be represented as an "adjacency dictionary", so that for every node `i` in the graph `self.edges[i]` is a dictionary of nodes that `i` is directly connected to. The value of the inner dictionary should be `1` for every edge that exists (the value of this entry doesn't matter, so we could technically use a dictionary of sets, but we use a dictionary of dictionaries to keep things a little bit more uniform and to allow for potential extensions e.g. to weighted graphs.)

Note that all vertices must exist in the dictionary. If a vertex has no outgoing edges, then it should still have an entry pointing to an empty dictionary.

To begin, implement the function `add_edges()`. It must modify the `self.edges` variable to add all edges passed as tuples in `edges_list`.

In [ ]:
class Graph:
    def __init__(self):
        """ Initialize with an empty edge dictionary. """
        self.edges = {}

    def add_edges(self, edges_list):
        """ Add a list of edges to the network. Use 1 to indicate the presence of an edge.

        Args:
            edges_list: list of (a, b) tuples, where a -> b is an edge to add

        Returns:
            self: instance of class (needed for testing)
        """
        for edge in edges_list:
            a, b = edge
            if a not in self.edges:
                self.edges[a] = {}
            if b not in self.edges:
                self.edges[b] = {}
            self.edges[a][b] = 1

        return self

    def adjacency_matrix(self):
        """ Compute an adjacency matrix form of the graph.
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjacency matrix
               for the graph (i.e., A[j,i] = 1 iff there is an edge i->j)
               NOTE: be sure you have this ordering correct!
            nodes: a list of nodes indicating the node key corresponding to each
                       index of the A matrix
        """
        nodes = list(self.edges.keys())
        node_to_index = {node: i for i, node in enumerate(nodes)}

        row, col = [], []
        for a in self.edges:
            for b in self.edges[a]:
                row.append(node_to_index[b])
                col.append(node_to_index[a])

        adjacency_matrix = sp.coo_matrix(([1] * len(row), (row, col)), shape=(len(nodes), len(nodes)))

        return adjacency_matrix, nodes



## Q2: Dijkstra's algorithm

Next, implement Dijkstra's single-source shortest path algorithm (with the simple case where the distance along any edge is assumed to be one).  You can refer to the [Wikipedia page on Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) for reference. The basic idea of the algorithm is to keep a priority queue of nodes ordered by distance from a source node.  At each step, we continually pop off the smallest element `i` in the queue, and update the distance of all successor nodes to have a distance of `1 + distance[i]`.

For the priority queue, you should use a [`heapdict`](https://github.com/DanielStutzbach/heapdict), which is a form of priority queue that allows you to change the priority of an element.

When called with source node `A`, the function should return a dictionary where the keys are all the nodes in the graph. For each key `B`:

- if `B` is reachable from `A` then the value must be the tuple `(distance, prev_node)`, where:
  - `distance` is the minimum number of hops from `A` to `B`, and
  - `prev_node` is the node immediately before `B` along one such shortest path
- if `B` is not reachable from `A`, then the value must be `(inf,None)`
- if `B == A`, then the value should be `(0, None)`

In [ ]:
import heapdict
import math
def shortest_path(g, source):
    """ Compute the single-source shorting path.

    This function uses Dijkstra's algorithm to compute the distance from
    source to all other nodes in the network.

    Args:
        g: Graph object
        source: source node

    Returns:
        dictionary of node: (distance, prev_node) values for each reachable node in the graph, where
              distance denotes the shortest distance from of node from source, and
              prev_node is the previous node on the shortest path from source to node.
            if node is unreachable, the value should be None
    """
    distances = {node: (math.inf, None) for node in g.edges}
    distances[source] = (0, None)

    pq = heapdict.heapdict()
    for node in g.edges:
        pq[node] = distances[node][0]

    while pq:
        current_node, current_distance = pq.popitem()

        if current_distance == math.inf:
            break

        for neighbor in g.edges[current_node]:
            distance = distances[current_node][0] + 1
            if distance < distances[neighbor][0]:
                distances[neighbor] = (distance, current_node)
                pq[neighbor] = distance

    return distances


## Q3: Adjacency matrix representation

Implement the adjacency matrix method of the Graph class.  This returns a matrix representing the adjacency of the graph (in scipy COO sparse format), as well as a list of nodes that indicate how the indices in this graph relate to the nodes in the network.

In order to complete this question in a manner that works on the Wikipedia graph, you must implement this function natively as a sparse matrix (i.e., you cannot construct a dense matrix and then convert that to a sparse matrix, but need to directly use the `sp.coo_matrix()` constructor).  The Wikipedia graph is 24K x 24K nodes, which (assuming 8 bytes per entry, would take up 4GB of memory.  While it's not impossible to do things this way at this scale (it quickly becomes infeasible for graphs that are even slightly larger), it's a very bad idea, and just allocating this much memory will take too long.

**Note the order of the axes of the output matrix.** This is important for calculating the PageRank.

In [ ]:
def adjacency_matrix(g):
    """ Compute an adjacency matrix form of the graph.
    Args:
        g: Graph object

    Returns: tuple (A, nodes)
        A: a sparse matrix in COO form that represents the adjacency matrix
           for the graph (i.e., A[j,i] = 1 iff there is an edge i->j)
           NOTE: be sure you have this ordering correct!
        nodes: a list of nodes indicating the node key corresponding to each
                   index of the A matrix
        """
    nodes = list(g.edges.keys())
    node_to_index = {node: i for i, node in enumerate(nodes)}

    row, col = [], []
    for a in g.edges:
        for b in g.edges[a]:
            row.append(node_to_index[b])
            col.append(node_to_index[a])

    adjacency_matrix = sp.coo_matrix(([1] * len(row), (row, col)), shape=(len(nodes), len(nodes)))

    return adjacency_matrix, nodes

Make sure your code works on the Wikipedia graph.  In our implementation, it takes about 4 seconds to generate this matrix from the Wikipedia graph (not including the time of the `read_graph` function).

## Q4: PageRank algorithm

Finally, implement the PageRank algorithm using the adjacency matrix representation. You should use the approach described in the ["Power method" section on the Wikipedia entry](https://en.wikipedia.org/wiki/PageRank#Power_method), which we also discussed in class.

This involves forming some initial uniform probability vector $x$, and repeatly multiplying it by the matrices:
\begin{equation}
x \gets \left(d P + \left(1-d\right)\frac{1}{n} E \right)x
\end{equation}
where $P$ is a transition matrix, $E$ is the matrix of all ones, and $d$ is the damping factor, and $n$ is the number of nodes. You get $P$ by normalizing $A$ so that all columns have sum 1.

*Note*: Here we are defining $d$ to be the "damping factor", which means we will randomly restart with probability $(1-d)$.

Recall that from the definition of PageRank, when we reach a "sink" node (a node with no outgoing edges), we randomly hop to any other node in the network, so that columns of $P$ that have no outgoing edges are set to the uniform distribution.  To be efficient, you'll also want to avoid explicitly forming the $E$ matrix, and should instead use the fact that $E = \mathbf{1}\mathbf{1}^T$ where $\mathbf{1}$ denotes a vector of all ones.  Use the fact that we can reorder matrix multiplication if associative (i.e., the fact the $A(BC)$ = $(AB)C$) to make this operation as fast as possible.

Your function should return a dictionary of nodes and their corresponding page rank.  For example, in the five-node graph from our test cases, we have the following results:
```
C: 0.324
B: 0.281
E: 0.188
D: 0.121
A: 0.085
```
As is intuitive, nodes B and C have higher page rank, as they are pointed to by more of the other nodes.

In [ ]:
def pagerank(g, d, iters):
    """ Compute the PageRank score for each node in the network.
    Args:
        g: Graph object
        d: PageRank damping factor d (restart with probability (1-d))
        iters: number of iterations to run

    Returns: dict ranks
        ranks: a dictionary of node:importance score, for each node in the
               network (larger score means higher rank)

    """
    A, nodes = g.adjacency_matrix()
    n = len(nodes)

    col_sum = np.array(A.sum(axis=0)).flatten()
    col_sum[col_sum == 0] = 1
    P = A / col_sum[:, np.newaxis]


    x = np.ones(n) / n

    for _ in range(iters):
        x = d * P.dot(x) + (1 - d) / n

    ranks = {nodes[i]: x[i] for i in range(n)}
    return ranks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

wikipedia_graph = Graph()
links = read_graph("/content/drive/MyDrive/Colab_Notebooks/wikipedia_small")
wikipedia_graph.add_edges(links)

page_ranks = pagerank(wikipedia_graph, 0.85, 20)

top_page_ranks = sorted(page_ranks.items(), key=lambda x: x[1], reverse=True)[:20]
for node, rank in top_page_ranks:
    print(f"{node}: {rank:.2e}")

Square_kilometer: 7.10e-02
Exempli_gratia: 5.82e-02
List_of_postal_codes_in_France: 5.79e-02
June_15: 3.45e-02
Roman_Catholic: 2.50e-02
Trungy: 2.31e-02
Football_(soccer): 2.26e-02
SI: 2.21e-02
List_of_countries_and_outlying_territories_by_area: 2.19e-02
United_States_of_America: 2.02e-02
Latin_language: 1.86e-02
Protestant: 1.77e-02
Doubs: 1.74e-02
Vikram_era: 1.72e-02
B3_DNA_binding_domain: 1.71e-02
BHLH: 1.71e-02
BZIP: 1.71e-02
Heat_Shock_Factor: 1.71e-02
Hypoxia_inducible_factors: 1.71e-02
JUND: 1.71e-02


Make sure your implementation works on the full Wikipedia graph (in our implementation, it takes 11 seconds to run, most of which is taken up by generating the adjacency matrix).  The top PageRank entires we get from our implementation are:

```
United_States     2.75e-3
2007              2.44e-3
2008              2.17e-3
Wikimedia_Commons 1.72e-3
United_Kingdom    1.59e-3
2006              1.54e-3
France            1.44e-3
Wiktionary        1.26e-3
Canada            1.09e-3
World_War_II      1.04e-3
2005              1.04e-3
List_of_Africa... 1.00e-3
Germany           0.95e-3
Europe            0.93e-3
English_language  0.90e-3
Geographic_coo... 0.89e-3
Latin             0.88e-3
Australia         0.87e-3
India             0.78e-3
Japan             0.78e-3
```

countries and years! A seemingly reasonable list of pages we may expect to be important.